In [1]:
! pip install mysql-connector-python

In [5]:
import pandas as pd
import numpy as np
import mysql.connector 

In [4]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="CampusX@21",
    database="dummy_project"
)

In [5]:
conn.is_connected()

True

In [6]:
# we will create an  function out of it
def connect_to_db():
    return mysql.connector.connect(
    host="localhost",
    user="root",
    password="CampusX@21",
    database="dummy_project")

In [8]:
connect_to_db().is_connected()

True

In [7]:
db = connect_to_db()
cursor = db.cursor(dictionary=True)

In [39]:
cursor

In [10]:
"select count(*) as total_suppliers from suppliers"

'select count(*) as total_suppliers from suppliers'

In [11]:
cursor.execute("select count(*) as total_suppliers from suppliers")

In [13]:
row = cursor.fetchone()

In [17]:
list(row.values())[0]

50

In [18]:
queries = {
"Total Suppliers": "SELECT COUNT(*) AS count FROM suppliers",

"Total Products": "SELECT COUNT(*) AS count FROM products",

"Total Categories Dealing": "SELECT COUNT(DISTINCT category) AS count FROM products",

"Total Sale Value (Last 3 Months)": """
SELECT ROUND(SUM(ABS(se.change_quantity) * p.price), 2) AS total_sale
FROM stock_entries se
JOIN products p ON se.product_id = p.product_id
WHERE se.change_type = 'Sale'
AND se.entry_date >= (
SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH) FROM stock_entries)
""",

"Total Restock Value (Last 3 Months)": """
SELECT ROUND(SUM(se.change_quantity * p.price), 2) AS total_restock
FROM stock_entries se
JOIN products p ON se.product_id = p.product_id
WHERE se.change_type = 'Restock'
AND se.entry_date >= (
SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH) FROM stock_entries)
""",

"Below Reorder & No Pending Reorders": """
SELECT COUNT(*) AS below_reorder
FROM products p
WHERE p.stock_quantity < p.reorder_level
AND p.product_id NOT IN (
SELECT DISTINCT product_id FROM reorders WHERE status = 'Pending')
"""
}

In [19]:
result={}
for label, query in queries.items():
    cursor.execute(query)
    row= cursor.fetchone()
    result[label]=list(row.values())[0]
    

In [21]:
result

{'Total Suppliers': 50,
 'Total Products': 200,
 'Total Categories Dealing': 5,
 'Total Sale Value (Last 3 Months)': 1529482.99,
 'Total Restock Value (Last 3 Months)': 6691663.1,
 'Below Reorder & No Pending Reorders': 14}

In [24]:
## Now lets create function out of it 

def get_basic_info(cursor):
    """
    Retrieve summary inventory and supply chain metrics.

    Args:
        cursor (mysql.connector.cursor.MySQLCursorDict): Cursor object with dictionary=True.

    Returns:
        dict: Dictionary of metric labels and their values.
    """

    queries = {
        "Total Suppliers": "SELECT COUNT(*) AS count FROM suppliers",

        "Total Products": "SELECT COUNT(*) AS count FROM products",

        "Total Categories Dealing": "SELECT COUNT(DISTINCT category) AS count FROM products",

        "Total Sale Value (Last 3 Months)": """
            SELECT ROUND(SUM(ABS(se.change_quantity) * p.price), 2) AS total_sale
            FROM stock_entries se
            JOIN products p ON se.product_id = p.product_id
            WHERE se.change_type = 'Sale'
              AND se.entry_date >= (
                  SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH) FROM stock_entries)
        """,

        "Total Restock Value (Last 3 Months)": """
            SELECT ROUND(SUM(se.change_quantity * p.price), 2) AS total_restock
            FROM stock_entries se
            JOIN products p ON se.product_id = p.product_id
            WHERE se.change_type = 'Restock'
              AND se.entry_date >= (
                  SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH) FROM stock_entries)
        """,

        "Below Reorder & No Pending Reorders": """
            SELECT COUNT(*) AS below_reorder
            FROM products p
            WHERE p.stock_quantity < p.reorder_level
              AND p.product_id NOT IN (
                  SELECT DISTINCT product_id FROM reorders WHERE status = 'Pending')
        """
    }

    results = {}
    for label, query in queries.items():
        cursor.execute(query)
        row = cursor.fetchone()
        # Since row is a dictionary, extract the single value by getting the first value in dict.values()
        results[label] = list(row.values())[0]

    return results



In [47]:
queries = {
        "Suppliers Contact Details": "SELECT supplier_name, contact_name, email, phone FROM suppliers",

        "Products with Supplier and Stock": """
            SELECT 
                p.product_name,
                s.supplier_name,
                p.stock_quantity,
                p.reorder_level
            FROM products p
            JOIN suppliers s ON p.supplier_id = s.supplier_id
            ORDER BY p.product_name ASC
        """,

        "Products Needing Reorder": """
            SELECT product_name, stock_quantity, reorder_level
            FROM products
            WHERE stock_quantity <= reorder_level
        """
    }

tables = {}
for label, query in queries.items():
    cursor.execute(query)
    tables[label] = cursor.fetchall()

In [49]:
tables

{'Suppliers Contact Details': [{'supplier_name': 'Anderson-Thompson',
   'contact_name': 'Bonnie Davis',
   'email': 'zacharysanchez@hotmail.com',
   'phone': '829.485.9853x0522'},
  {'supplier_name': 'Rowland Ltd',
   'contact_name': 'Beth Stevens',
   'email': 'joshua60@yahoo.com',
   'phone': '(779)942-0726'},
  {'supplier_name': 'Baxter-Meadows',
   'contact_name': 'Lisa Lewis',
   'email': 'andersonchristina@yahoo.com',
   'phone': '449-766-7325'},
  {'supplier_name': 'Wilson, Graham and Williams',
   'contact_name': 'David Martinez',
   'email': 'abrown@hotmail.com',
   'phone': '+1-653-827-5215x266'},
  {'supplier_name': 'Smith, Kennedy and Moreno',
   'contact_name': 'Victoria Gonzalez',
   'email': 'mary30@williams-moore.com',
   'phone': '891-859-2775x35297'},
  {'supplier_name': 'Middleton LLC',
   'contact_name': 'Megan Miller',
   'email': 'kelseywilliams@gmail.com',
   'phone': '001-724-731-6199x4596'},
  {'supplier_name': 'Evans Inc',
   'contact_name': 'Danielle Moore',

In [50]:
def get_additonal_tables(cursor):
    queries = {
        "Suppliers Contact Details": "SELECT supplier_name, contact_name, email, phone FROM suppliers",

        "Products with Supplier and Stock": """
            SELECT 
                p.product_name,
                s.supplier_name,
                p.stock_quantity,
                p.reorder_level
            FROM products p
            JOIN suppliers s ON p.supplier_id = s.supplier_id
            ORDER BY p.product_name ASC
        """,

        "Products Needing Reorder": """
            SELECT product_name, stock_quantity, reorder_level
            FROM products
            WHERE stock_quantity <= reorder_level
        """
    }

    tables = {}
    for label, query in queries.items():
        cursor.execute(query)
        tables[label] = cursor.fetchall()

    return tables

In [51]:
get_additonal_tables(cursor)

{'Suppliers Contact Details': [{'supplier_name': 'Anderson-Thompson',
   'contact_name': 'Bonnie Davis',
   'email': 'zacharysanchez@hotmail.com',
   'phone': '829.485.9853x0522'},
  {'supplier_name': 'Rowland Ltd',
   'contact_name': 'Beth Stevens',
   'email': 'joshua60@yahoo.com',
   'phone': '(779)942-0726'},
  {'supplier_name': 'Baxter-Meadows',
   'contact_name': 'Lisa Lewis',
   'email': 'andersonchristina@yahoo.com',
   'phone': '449-766-7325'},
  {'supplier_name': 'Wilson, Graham and Williams',
   'contact_name': 'David Martinez',
   'email': 'abrown@hotmail.com',
   'phone': '+1-653-827-5215x266'},
  {'supplier_name': 'Smith, Kennedy and Moreno',
   'contact_name': 'Victoria Gonzalez',
   'email': 'mary30@williams-moore.com',
   'phone': '891-859-2775x35297'},
  {'supplier_name': 'Middleton LLC',
   'contact_name': 'Megan Miller',
   'email': 'kelseywilliams@gmail.com',
   'phone': '001-724-731-6199x4596'},
  {'supplier_name': 'Evans Inc',
   'contact_name': 'Danielle Moore',

In [2]:
def add_new_manual_id(cursor, db, p_name , p_category , p_price , p_stock , p_reorder, p_supplier):
    proc_call= "call AddNewProductManualID(%s, %s, %s ,%s ,%s, %s)"
    params= (p_name , p_category , p_price , p_stock , p_reorder, p_supplier)
    cursor.execute(proc_call, params)
    db.commit()

In [12]:
def get_categories(cursor):
    cursor.execute("select Distinct category  from products  order by category  asc")
    rows= cursor.fetchall()
    return [row["category"] for row in rows]

In [13]:
get_categories(cursor)

['Clothing', 'Electronics', 'Furniture', 'Groceries', 'Toys']

In [15]:
def get_suppliers(cursor):
    cursor.execute("select supplier_id , supplier_name from suppliers order by  supplier_name asc")
    return cursor.fetchall()

In [19]:
suppliers= get_suppliers(cursor)

In [20]:
supplier_ids=[s["supplier_id"] for s in suppliers]
supplier_names=[s["supplier_name"] for s in suppliers]

In [22]:
supplier_names

['Anderson-Thompson',
 'Armstrong-Vance',
 'Barker, White and Carson',
 'Barrett Ltd',
 'Baxter-Meadows',
 'Charles Inc',
 'Clark Group',
 'Douglas Ltd',
 'Elliott-Ayers',
 'Evans Inc',
 'Franklin, Kane and Price',
 'Freeman-Gordon',
 'Gallagher-Miller',
 'Gomez PLC',
 'Hall-Brown',
 'Harris-Cummings',
 'Henderson LLC',
 'Hensley-Branch',
 'Hudson Inc',
 'Johnson-Bass',
 'Kaufman Ltd',
 'Lawrence, Garcia and Hernandez',
 'Lloyd and Sons',
 'Mann-Marshall',
 'Mendoza-Jones',
 'Middleton LLC',
 'Miller-Martinez',
 'Moody-Vang',
 'Morgan Inc',
 'Morgan-Andrews',
 'Moss-Evans',
 'Newton, Valencia and Carr',
 'Ortega-Mahoney',
 'Patrick, Walter and Harrison',
 'Perez, Price and Wallace',
 'Reynolds-Phillips',
 'Rogers-Greene',
 'Rowe PLC',
 'Rowland Ltd',
 'Smith, Kennedy and Moreno',
 'Stewart, Williams and Cox',
 'Taylor-Love',
 'Tran LLC',
 'Tucker-Arnold',
 'Turner-Davis',
 'Vega, Cook and Miller',
 'Williams Ltd',
 'Wilson, Graham and Williams',
 'Wong Group',
 'Young, Browning and War

In [26]:
print(lambda x: supplier_names[supplier_ids.index(x)])

<function <lambda> at 0x0000021FD686A2A0>


In [33]:
def get_pending_reorders(cursor):
    cursor.execute("""
    select r.reorder_id , p.product_name
    from reorders as r join products as p 
    on r.product_id= p.product_id
    """)
    return cursor.fetchall()

In [34]:
get_pending_reorders(cursor)

[{'reorder_id': 1, 'product_name': 'Someone Shirt'},
 {'reorder_id': 2, 'product_name': 'Six Table'},
 {'reorder_id': 3, 'product_name': 'Space Toy'},
 {'reorder_id': 4, 'product_name': 'Blue Device'},
 {'reorder_id': 5, 'product_name': 'Mouth Shirt'},
 {'reorder_id': 6, 'product_name': 'School Table'},
 {'reorder_id': 7, 'product_name': 'Four Shirt'},
 {'reorder_id': 8, 'product_name': 'Fast Shirt'},
 {'reorder_id': 9, 'product_name': 'Character Table'},
 {'reorder_id': 10, 'product_name': 'Fact Device'},
 {'reorder_id': 11, 'product_name': 'Return Table'},
 {'reorder_id': 12, 'product_name': 'Scene Table'},
 {'reorder_id': 13, 'product_name': 'Mission Snack'},
 {'reorder_id': 14, 'product_name': 'Old Shirt'},
 {'reorder_id': 15, 'product_name': 'Within Toy'},
 {'reorder_id': 16, 'product_name': 'Lead Toy'},
 {'reorder_id': 17, 'product_name': 'Guy Device'},
 {'reorder_id': 18, 'product_name': 'Bag Table'},
 {'reorder_id': 19, 'product_name': 'However Table'},
 {'reorder_id': 20, 'pro